In [1]:
import pymongo
from pymongo import MongoClient
import json
import numpy as np
import datetime
import re
import geopy
import winsound
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
 


In [2]:
# Connect to database
conn     = MongoClient('localhost', 27017)
db       = conn.bigdata
db_clean = conn.bigdata_clean

# Collection Name

# traffic_accident
clean_name = 'clean_accident'
data      = db.traffic_accident

id_data = db_clean.clean_accident
collection = db_clean[clean_name]


# # kecelakaan
# clean_name = 'clean_kecelakaan'
# data      = db.kecelakaan

# id_data = db_clean.clean_kecelakaan
# collection = db_clean[clean_name]


# # tabrakan
# clean_name = 'clean_tabrakan'
# data      = db.tabrakan

# id_data = db_clean.clean_tabrakan
# collection = db_clean[clean_name]



# # # terjadi_kecelakaan
# clean_name = 'clean_terjadi_kecelakaan'
# data      = db.terjadi_kecelakaan

# id_data = db_clean.clean_terjadi_kecelakaan
# collection = db_clean[clean_name]

In [3]:
# Array data
count = 0
arr_id = []
for id in id_data.find():
    count +=1
    arr_id.append(id["id"])
    
# print("Total : {}".format(count))

In [4]:
with open('kabupaten.json') as f:
    list_kab = json.load(f)
    kab      = [item.lower() for item in list_kab]

# print(kab)

In [5]:
with open('nama_kota.json', encoding="utf8") as f:
    list_kab  = json.load(f)
    city      = [item['name'].lower() for item in list_kab]
    
# print(city)

In [6]:
def fun_get_koordinat(location):
    try:
            #Ubah Nama kota ke koordinat
            geolocator = Nominatim(user_agent="specify_your_app_name_here")
            location = geolocator.geocode(location)
            
            if(location is not None and location.latitude is not None):
                lat = location.latitude
                lng = location.longitude
                koordinat = (lat, lng)
            else:
                koordinat = None
        
            return koordinat

    except GeocoderTimedOut:
        print("Server Time Out, retry clean data...................")
        time.sleep(5)
        frequency = 2000  # Set Frequency To 2500 Hertz
        duration = 500  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)
        winsound.Beep(frequency, duration)
        fun_input_data(item, lowcase_text )

In [7]:
text = "#Bandung nasib naas bagi pengemudi sepeda motor".lower()
word = "jakarta"

for i in kab:
    word = str(i)
    if(word in text):
        print(word)


bandung


In [8]:
def fun_input_data(item, lowcase_text):
    data = {}
    
    user_loc = item["user"]
    geo = item["geo"]
    
#     if(item["full_text"] is not None):
#         lowcase_text = item["full_text"].lower()
#     elif(item["text"] is not None):
#         lowcase_text = item["text"].lower()
    
    data['id']          = item["id"]
    data['created_at']  = item["created_at"]
    data['text']        = lowcase_text  
    
    
    CITY_LOC = ''
    #Get Location tweet post
    for word in kab:
        lowcase_text = lowcase_text.lower()
        if(word in lowcase_text):
            CITY_LOC  = word
#             print("Kota di text :"+CITY_LOC)
    
        elif(user_loc is not None):
            CITY_LOC  = user_loc["location"].lower()
#             print("Lokasi       :"+CITY_LOC)
    
    
    result    = re.findall(r'^[^,]+(?=,)',CITY_LOC)  #Get Nama Kota sebelum tanda koma

    dlt_kota   = re.sub("kota ",'',"{}".format(result)) #Edit nama kota
    dlt_dki    = re.sub("dki ",'',"{}".format(dlt_kota))
    
    if(item["geo"] is not None):
        koordinat = geo["coordinates"]
    
    elif(CITY_LOC in kab or result in kab or dlt_dki in kab or CITY_LOC in city):
        if(CITY_LOC in kab or CITY_LOC in city):
            koordinat = fun_get_koordinat(CITY_LOC)
            
        elif(result in kab):
            koordinat = fun_get_koordinat(result)
        
        elif(dlt_dki in kab):
            koordinat = fun_get_koordinat(dlt_dki)
    else:
        koordinat = None   
    
    data['location']    = koordinat


    file.write(json.dumps(data) + '\n')
    collection.insert(data)
    print(data)
    

In [9]:
count_tweet = 0
currentDT = datetime.datetime.now()
with open('hasil/Clean/{}_Tanggal-{}-{}.json'.format(clean_name,currentDT.month,currentDT.day), 'w+') as file:
    for item in data.find():
        if(item["text"] is not None):
            lowcase_text = item["text"].lower()
        else:
            lowcase_text = "rt @"
            
        if("rt @" not in lowcase_text):
            if(item['id'] not in arr_id):
                arr_id.append(item["id"])
                
                count_tweet +=1
                
                if("ancam" not in lowcase_text or 
                   "dipaksa" not in lowcase_text or
                   "rokok" not in lowcase_text or
                   "bantu share" not in lowcase_text or
                   "repost" not in lowcase_text or
                   "sering" not in lowcase_text or
                   "keren" not in lowcase_text or
                   "diperingatkan" not in lowcase_text or
                   "peringatan" not in lowcase_text or
                   "sumbangan" not in lowcase_text or
                   "menabrakan" not in lowcase_text or
                   "mainan" not in lowcase_text or
                   "mencegah" not in lowcase_text or
                   "melindungi" not in lowcase_text or
                   "diperhatikan" not in lowcase_text or
                   "diperingatkan" not in lowcase_text or
                   "warning" not in lowcase_text or
                   "jas hujan" not in lowcase_text or
                   "memperhatikan" not in lowcase_text or
                   "olahraga" not in lowcase_text or
                   "olah raga" not in lowcase_text or
                   "rawan" not in lowcase_text or
                   "berubah" not in lowcase_text or
                   "dokter" not in lowcase_text or
                   "baca" not in lowcase_text or
                   "kapanpun" not in lowcase_text or
                   "mantan" not in lowcase_text or
                   "jomblo" not in lowcase_text or
                   "memperbaiki" not in lowcase_text or
                   "hadir" not in lowcase_text or
                   "pemakaman" not in lowcase_text or
                   "menabrakan" and "ganti rugi" not in lowcase_text or
                   "ganti rugi" and "diancam" not in lowcase_text or
                   "kecelakaan" and "hati" not in lowcase_text):
                    
                    fun_input_data(item, lowcase_text) 
                
print("\n Total Data : {}".format(count_tweet))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


{'id': 1126637958872883200, 'created_at': 'Thu May 09 23:59:55 +0000 2019', 'text': '(4/4) and started helping the man push his car to the side of the road in the middle of rush hour traffic. and thus… https://t.co/xepfjes7vd', 'location': None, '_id': ObjectId('5cd5c1d881fa970ee1e0d083')}
{'id': 1126637836961292288, 'created_at': 'Thu May 09 23:59:26 +0000 2019', 'text': "@txdotodessa do ya'll have traffic backup in the penwell area? accident? eta?", 'location': None, '_id': ObjectId('5cd5c1d881fa970ee1e0d084')}
{'id': 1126637804111433728, 'created_at': 'Thu May 09 23:59:19 +0000 2019', 'text': 'accident cleared in #portland on i 5 sb at broadway, stopped traffic back to i-405, delay of 4 mins #pdxtraffic https://t.co/cq6bnj5b3o', 'location': [45.53509, -122.667], '_id': ObjectId('5cd5c1d881fa970ee1e0d085')}
{'id': 1126637424749236224, 'created_at': 'Thu May 09 23:57:48 +0000 2019', 'text': 'the left lane is blocked i-110 south at harding boulevard due to an accident. congestion is mi

{'id': 1126626666917322752, 'created_at': 'Thu May 09 23:15:03 +0000 2019', 'text': 'any news on hwy 60 closure due to mayor traffic accident in polk county fl', 'location': (27.9014133, -81.5859099), '_id': ObjectId('5cd5c1da81fa970ee1e0d0b6')}
{'id': 1126626416072830976, 'created_at': 'Thu May 09 23:14:03 +0000 2019', 'text': 'accident cleared in #eastside on 610 e lp sb at the port of houston, stop and go traffic back to market, delay of 8 mins #traffic', 'location': [29.73145, -95.26613], '_id': ObjectId('5cd5c1da81fa970ee1e0d0b7')}
{'id': 1126626278478622720, 'created_at': 'Thu May 09 23:13:31 +0000 2019', 'text': 'accident, shoulder blocked in #stockton on hwy 99 sb before hwy 26, slow traffic back to hwy 88, delay of 1 min #traffic', 'location': [37.96992, -121.2457], '_id': ObjectId('5cd5c1da81fa970ee1e0d0b8')}
{'id': 1126626268156448768, 'created_at': 'Thu May 09 23:13:28 +0000 2019', 'text': 'accident blocks a right lane and a right shoulder. in #southside on 610 s lp eb at s

{'id': 1126623064471396352, 'created_at': 'Thu May 09 23:00:44 +0000 2019', 'text': 'traffic 🚗🚗🚗: heavy traffic on cte (towards aye) between yio chu kang rd exit and pie(changi) exit.\n\naccident on pat… https://t.co/jlksbhhh8i', 'location': (1.3408528, 103.878446863736), '_id': ObjectId('5cd5c1e381fa970ee1e0d0d9')}
{'id': 1126623058234609664, 'created_at': 'Thu May 09 23:00:43 +0000 2019', 'text': 'accident cleared in #mtlaurel on i-295 sb between rt-73 and rt-70, jammed back to rt-73, delay of 6 mins #traffic https://t.co/btegsrlpfo', 'location': [39.93484, -74.96602], '_id': ObjectId('5cd5c1e381fa970ee1e0d0da')}
{'id': 1126623012130885632, 'created_at': 'Thu May 09 23:00:32 +0000 2019', 'text': "traffic accident fatalities drop thanks to gov't measures https://t.co/2ru3hpjvlu", 'location': (41.0096334, 28.9651646), '_id': ObjectId('5cd5c1e481fa970ee1e0d0db')}
{'id': 1126622997316546560, 'created_at': 'Thu May 09 23:00:28 +0000 2019', 'text': 'https://t.co/v6z5oknhtk single car accid

Server Time Out, retry clean data...................
Server Time Out, retry clean data...................
{'id': 1126615311027806208, 'created_at': 'Thu May 09 22:29:56 +0000 2019', 'text': 'there’s a traffic accident i have to mention', 'location': (40.8401984, -73.9402214), '_id': ObjectId('5cd5c1f481fa970ee1e0d110')}
{'id': 1126615311027806208, 'created_at': 'Thu May 09 22:29:56 +0000 2019', 'text': 'there’s a traffic accident i have to mention', 'location': None, '_id': ObjectId('5cd5c1f481fa970ee1e0d111')}
{'id': 1126615311027806208, 'created_at': 'Thu May 09 22:29:56 +0000 2019', 'text': 'there’s a traffic accident i have to mention', 'location': None, '_id': ObjectId('5cd5c1f481fa970ee1e0d112')}
{'id': 1126615292367470592, 'created_at': 'Thu May 09 22:29:51 +0000 2019', 'text': 'accident in #bna on i 40 eb before briley pkwy #nashville #traffic https://t.co/ru1w1urskb', 'location': [36.1442, -86.69739], '_id': ObjectId('5cd5c1f481fa970ee1e0d113')}
{'id': 1126615290773635072, 'cr

...

{'id': 1125636421187002368, 'created_at': 'Tue May 07 05:40:10 +0000 2019', 'text': "avoid southern bypass if ur in a hurry esp btw ngong rd and lang'ata rd intersections...crazy traffic\na lorry had a… https://t.co/chrdxilaty", 'location': (-1.2832533, 36.8172449), '_id': ObjectId('5cd5c36181fa970ee1e0e8b8')}
{'id': 1125636332490039296, 'created_at': 'Tue May 07 05:39:49 +0000 2019', 'text': '#trafficupdate | 09:36\n\n#accident on smbzr opposite jumeirah village, resulting in traffic delays. please drive safely and carefully.', 'location': None, '_id': ObjectId('5cd5c36181fa970ee1e0e8b9')}
{'id': 1125636325955317760, 'created_at': 'Tue May 07 05:39:48 +0000 2019', 'text': '"@austynzogs: @gidi_traffic 06h32: accident at sango toll gate area, causing traffic on lagos/abeokuta expressway #ridewithgidicab', 'location': None, '_id': ObjectId('5cd5c36181fa970ee1e0e8ba')}
{'id': 1125635985277116416, 'created_at': 'Tue May 07 05:38:26 +0000 2019', 'text': "avoid southern bypass if ur in a hur

{'id': 1125612485900369920, 'created_at': 'Tue May 07 04:05:04 +0000 2019', 'text': 'temp traffic lights today at deadaway bridge #maldon rd btwn #latchingdon &amp; #mundon for urgent parapet repairs afte… https://t.co/bmhhkmzsri', 'location': (51.7967047, 0.579338882096011), '_id': ObjectId('5cd5c36381fa970ee1e0e8d6')}
{'id': 1125612098418094080, 'created_at': 'Tue May 07 04:03:31 +0000 2019', 'text': 'yes i look at the accident after ive waited in traffic to pass it, its my turn.', 'location': None, '_id': ObjectId('5cd5c36381fa970ee1e0e8d7')}
{'id': 1125611473886015488, 'created_at': 'Tue May 07 04:01:02 +0000 2019', 'text': 'nothing makes me more mad than nosy ppl looking at a accident on the highway slowing down traffic', 'location': None, '_id': ObjectId('5cd5c36381fa970ee1e0e8d8')}
{'id': 1125610921072545792, 'created_at': 'Tue May 07 03:58:51 +0000 2019', 'text': '5月11日（土）から「春の全国交通安全運動」が始まります。5月は小学生の死傷事故がもっとも多い月です。特に下校が始まる15時以降に多く発生しています。今月も安全運転でいきましょう！ https://t.co/fqmk0gdzvt 

{'id': 1125584752797294592, 'created_at': 'Tue May 07 02:14:52 +0000 2019', 'text': '@sazeracla i read a story a few years ago about residents in the 405 overpass adjacent area posting fake accident/t… https://t.co/asy5iu715z', 'location': (34.0536909, -118.2427666), '_id': ObjectId('5cd5c36d81fa970ee1e0e8f7')}
{'id': 1125583673967857664, 'created_at': 'Tue May 07 02:10:34 +0000 2019', 'text': '@blrcitytraffic huge truck accident at agara flyover. extreme traffic. may increase even more.', 'location': None, '_id': ObjectId('5cd5c36d81fa970ee1e0e8f8')}
{'id': 1125583663855312896, 'created_at': 'Tue May 07 02:10:32 +0000 2019', 'text': 'traffic at a standstill outside of a dusty blanchetown due an accident between a truck and caravan. the latest in 7… https://t.co/xgyxbpf3y8', 'location': None, '_id': ObjectId('5cd5c36d81fa970ee1e0e8f9')}
{'id': 1125580506505064448, 'created_at': 'Tue May 07 01:57:59 +0000 2019', 'text': '5/6/2019@9:54 pm: traffic accident with injury at 3690 wheeler rd 

Server Time Out, retry clean data...................
{'id': 1125560611801141248, 'created_at': 'Tue May 07 00:38:56 +0000 2019', 'text': '@cjscalia @wsjopinion she uses every cheap prosecutor trick when questioning a witness — assume facts, misconstrue… https://t.co/uurbgpawpx', 'location': (34.0536909, -118.2427666), '_id': ObjectId('5cd5c37581fa970ee1e0e91c')}
{'id': 1125560611801141248, 'created_at': 'Tue May 07 00:38:56 +0000 2019', 'text': '@cjscalia @wsjopinion she uses every cheap prosecutor trick when questioning a witness — assume facts, misconstrue… https://t.co/uurbgpawpx', 'location': None, '_id': ObjectId('5cd5c37581fa970ee1e0e91d')}
{'id': 1125560368808517632, 'created_at': 'Tue May 07 00:37:58 +0000 2019', 'text': '5/6/2019@8:37 pm: traffic accident no injury at 3102 peach orchard rd augusta ga https://t.co/skrjoccxdk', 'location': None, '_id': ObjectId('5cd5c37581fa970ee1e0e91e')}
{'id': 1125559937625743360, 'created_at': 'Tue May 07 00:36:15 +0000 2019', 'text': 'accid

Server Time Out, retry clean data...................
{'id': 1125551326916747264, 'created_at': 'Tue May 07 00:02:02 +0000 2019', 'text': "#uber share your location and let your loved ones know that you've been involved in an accident because your uber d… https://t.co/qdbdp7yrdn", 'location': (51.5073219, -0.1276474), '_id': ObjectId('5cd5c38181fa970ee1e0e93e')}
{'id': 1125551326916747264, 'created_at': 'Tue May 07 00:02:02 +0000 2019', 'text': "#uber share your location and let your loved ones know that you've been involved in an accident because your uber d… https://t.co/qdbdp7yrdn", 'location': None, '_id': ObjectId('5cd5c38181fa970ee1e0e93f')}
{'id': 1125551185698742272, 'created_at': 'Tue May 07 00:01:29 +0000 2019', 'text': 'accident, right lane blocked in #queens on the l.i.e. wb at x32, stop and go traffic back to x34, delay of 6 mins #traffic', 'location': [40.76411, -73.72407], '_id': ObjectId('5cd5c38181fa970ee1e0e940')}

 Total Data : 6306


In [11]:
import winsound
frequency = 2000  # Set Frequency To 2500 Hertz
duration = 500  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)